<a href="https://colab.research.google.com/github/dinisrferreira/Pseudodiagnosticity-in-a-continuous-learning-environment-Reis-2020-/blob/main/DiagnosticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load data from Google Drive
folder_path = '/content/drive/My Drive/Data for DiagnosticNet/'
wicked_train_set = pd.read_csv(f'{folder_path}wicked_train_set_20240801_214722.csv')
wicked_test_set = pd.read_csv(f'{folder_path}wicked_test_set_20240801_214722.csv')
kind_train_set = pd.read_csv(f'{folder_path}kind_train_set_20240801_214722.csv')
kind_test_set = pd.read_csv(f'{folder_path}kind_test_set_20240801_214722.csv')

In [ ]:
def prepare_data(df):
    X = df['input'].apply(eval).values.tolist()
    full_values = df['full_values'].apply(eval).values.tolist()
    y = df['label'].values
    initial_choices = df['initial_choice'].values  # Get initial choices from the DataFrame
    golden_labels = df['golden_label'].apply(eval).values.tolist()

    X_tensor = torch.tensor(X, dtype=torch.float32)
    full_values_tensor = torch.tensor(full_values, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    initial_choices_tensor = torch.tensor(initial_choices, dtype=torch.long)  # Convert to tensor
    golden_labels_tensor = torch.tensor(golden_labels, dtype=torch.float32)  # Convert to tensor

    return TensorDataset(X_tensor, full_values_tensor, y_tensor, initial_choices_tensor, golden_labels_tensor)  # Include golden_labels in the TensorDataset

In [ ]:
train_data_wicked = prepare_data(wicked_train_set)
test_data_wicked = prepare_data(wicked_test_set)
train_data_kind = prepare_data(kind_train_set)
test_data_kind = prepare_data(kind_test_set)

batch_size = 32
train_loader_wicked = DataLoader(train_data_wicked, batch_size=batch_size, shuffle=True)
test_loader_wicked = DataLoader(test_data_wicked, batch_size=batch_size, shuffle=False)
train_loader_kind = DataLoader(train_data_kind, batch_size=batch_size, shuffle=True)
test_loader_kind = DataLoader(test_data_kind, batch_size=batch_size, shuffle=False)

In [ ]:
class DiagnosticNet(nn.Module):
    def __init__(self):
        super(DiagnosticNet, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 4)  # Output 4 probabilities, one for each cell

    def forward(self, x, initial_choices):
        x = torch.relu(self.fc1(x))
        logits = self.fc2(x)

        # Create a mask to zero out the probability of the initial cell
        mask = torch.ones_like(logits)
        for i, initial_choice in enumerate(initial_choices):
            mask[i, initial_choice] = 0

        masked_logits = logits * mask
        # Apply softmax to get probability distribution
        probs = torch.softmax(masked_logits, dim=1)

        return probs

In [ ]:
def determine_strategy(initial_choice, second_choice):
    row_initial, col_initial = divmod(initial_choice, 2)
    row_second, col_second = divmod(second_choice, 2)

    if row_initial == row_second:
        return 'row'
    elif col_initial == col_second:
        return 'column'
    else:
        return 'diagonal'

In [ ]:
# Define the loss function
criterion = nn.KLDivLoss(reduction='batchmean')

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        strategy_counts = {'row': 0, 'column': 0, 'diagonal': 0}

        for inputs, full_values, labels, initial_choices, golden_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs, initial_choices)

            # Compute the loss using KLDivLoss
            loss = criterion(outputs.log(), golden_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Track strategy usage
            _, predicted_choices = torch.max(outputs, 1)
            for i in range(len(inputs)):
                initial_choice = initial_choices[i].item()
                second_choice = predicted_choices[i].item()
                chosen_strategy = determine_strategy(initial_choice, second_choice)
                strategy_counts[chosen_strategy] += 1

        avg_loss = running_loss / len(train_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
        print(f"Strategy usage - Row: {strategy_counts['row']}, Column: {strategy_counts['column']}, Diagonal: {strategy_counts['diagonal']}")
        print('-' * 50)

In [ ]:
# Instantiate the model, optimizer, and scheduler
model = DiagnosticNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

Por alguma razão escolhas diagonais acontecem imenso no ambiente wicked (pode ser algo relacionado com os dados idk). No ambiente kind pelo menos descem at some point. Compreensivelmente a loss começa baixa no wicked por isso tenho de evitar aprendizagem muito rápida.

In [ ]:
# Train the model for wicked and kind environments
print("Training on wicked environment:")
train_model(model, train_loader_wicked, criterion, optimizer, num_epochs=50)
print
print("Training on kind environment:")
train_model(model, train_loader_kind, criterion, optimizer, num_epochs=50)

Training on wicked environment:
Epoch [1/50], Loss: 0.6865
Strategy usage - Row: 208, Column: 398, Diagonal: 194
--------------------------------------------------
Epoch [2/50], Loss: 0.6744
Strategy usage - Row: 70, Column: 364, Diagonal: 366
--------------------------------------------------
Epoch [3/50], Loss: 0.6619
Strategy usage - Row: 185, Column: 308, Diagonal: 307
--------------------------------------------------
Epoch [4/50], Loss: 0.6490
Strategy usage - Row: 267, Column: 200, Diagonal: 333
--------------------------------------------------
Epoch [5/50], Loss: 0.6346
Strategy usage - Row: 359, Column: 105, Diagonal: 336
--------------------------------------------------
Epoch [6/50], Loss: 0.6179
Strategy usage - Row: 371, Column: 76, Diagonal: 353
--------------------------------------------------
Epoch [7/50], Loss: 0.5986
Strategy usage - Row: 371, Column: 61, Diagonal: 368
--------------------------------------------------
Epoch [8/50], Loss: 0.5781
Strategy usage - Row